In [1]:
import numpy as np
import pandas as pd
from scipy.stats import uniform
import cbadc as cb
from itertools import product
from numpy.linalg import LinAlgError
from scipy.stats import qmc
import time

In [ ]:
def generate_dataset(form, order, implementation, t_min, num_samples, min_snr, parameter_ranges, discrete_params):
    """
    Generate a dataset based on the selected form, order, implementation
    
    Args:
    - form (str): The (continuous) form of the filter ('FB', 'FF')
    - order (int): The order of the filter (2, 3, 4)
    - implementation (str): The implementation type ('Active-RC', 'Gm-C')
    - t_min (float): Time in seconds to execute the function
    - num_samples (int): Initial number of samples to run LHS
    - min_snr (float): Minimum SNR accepted.
    - parameter_ranges (list): List with the range of each parameter
    - discrete_parameters (list): List with the discrete parameters and each values
    
    Returns:
    - dataset (pd.DataFrame): A DataFrame containing the generated dataset,
                              named: dataset_form_order_implementation.csv
    """
    
    start_time = time.time() # record start time
    
    # initialize lists to store the parameters and corresponding SNR values after simulation:
    snr_values = []
    all_params = []
    
    while (time.time() - start_time) < t_min:
        # Latin Hypercube Sampling (LHS) to sample the parameter space:
        lhs_samples = lhs_sampling(parameter_ranges, num_samples, discrete_params)

        # iterate through each set of parameters from the LHS samples:
        for sample in lhs_samples:
            # map the sampled values to parameter names:
            if implementation == 'Active-RC':
                params = {'Bw': sample[0], 'osr': int(sample[1]), 'nlev': int(sample[2]), 'Hinf': sample[3], 
                          'Cint': sample[4], 'gm': sample[5], 'Ro': sample[6], 'Co': sample[7]}
            elif implementation == 'Gm-C':
                params = {'Bw': sample[0], 'osr': int(sample[1]), 'nlev': int(sample[2]), 'Hinf': sample[3], 
                          'Cint': sample[4], 'Ro': sample[5], 'Cp': sample[6], 'v_n': sample[7], 
                          'slew_rate': sample[8], 'output_swing': sample[9]}

            # apply parameter relationships filters to ensure validity (not implemented yet):
            if not filter_parameter_relationships(params, implementation):
                continue # skip this sample if it doesn't pass the filter

            # simulate the circuit and get the SNR value:
            snr = simulate_circuit(form, order, implementation, params)

            # store the the parameters and corresponding SNR value:
            if snr is not None and np.isfinite(snr) and snr > min_snr:
                snr_values.append(snr)
                all_params.append(params)
    
    # create a DataFrame and save it as CSV:
    dataset = pd.DataFrame(all_params)
    dataset['SNR'] = snr_values
    filename = f"datasets/dataset_{form}_{order}_{implementation}.csv"
    dataset.to_csv(filename, index=False)
    print(f"\n\tDataset saved as {filename}\n")
    
    return dataset

In [3]:
def lhs_sampling(parameter_ranges, num_samples, discrete_params):
    """
    Latin Hypercube Sampling (LHS) for a given set of parameter ranges,
    handling both continuous and discrete parameters

    Args:
    - parameter_ranges (list): List of tuples [(min, max), ...] defining the range for each parameter
    - num_samples (int): Number of samples to generate
    - discrete_params (list): List of tuples [(index, values)] where:
        - index: The position of the discrete parameter in parameter_ranges
        - values: A list of possible discrete values

    Returns:
    - samples (list): A NumPy array of shape (num_samples, len(parameter_ranges)) containing the generated samples
    """

    # separate continuous and discrete parameters:
    continuous_params = [p for i, p in enumerate(parameter_ranges) if i not in [d[0] for d in discrete_params]]
    discrete_indices = {d[0]: d[1] for d in discrete_params}

    # create an LHS design only for continuous parameters:
    sampler = qmc.LatinHypercube(d=len(continuous_params)) # d = number of continuous parameters
    sample_cont = sampler.random(num_samples) # generate LHS samples in [0,1] range
    
    # rescale LHS samples to match the specified ranges for continuous parameters:
    cont_scaled = qmc.scale(sample_cont, 
                            [p[0] for p in continuous_params], # min values
                            [p[1] for p in continuous_params]) # max values
    
    # initialize the result matrix:
    samples = np.zeros((num_samples, len(parameter_ranges)))

    # insert continuous values into the result matrix:
    cont_idx = 0 # index to track continuous parameters in cont_scaled
    for i, p in enumerate(parameter_ranges):
        if i in discrete_indices: # skip discrete parameters
            continue
        samples[:, i] = cont_scaled[:, cont_idx] # assign LHS-sampled values
        cont_idx += 1

    # insert discrete parameters using random selection with replacement:
    for idx, values in discrete_indices.items():
        samples[:, idx] = np.random.choice(values, size=num_samples, replace=True)

    return samples

In [4]:
# LHS test:

parameter_ranges = [
            (1e3, 1e9), # Bw (Bandwidth)
            [8, 16, 32, 64, 128, 256], # osr (Oversampling Ratio) - Discrete
            [2, 3, 4, 5], # nlev (Number of Levels) - Discrete
            (1e-3, 1e1), # Hinf (Infinity Gain)
            (1e-12, 1e-6), # Cint (Integration Capacitance)
            (1e-3, 1e-1), # gm (Transconductance)
            (1e3, 1e6), # Ro (Output Resistance)
            (1e-12, 1e-6), # Co (Output Capacitance)
        ]
discrete_params = [
        (1, [8, 16, 32, 64, 128, 256]), # osr (discrete)
        (2, [2, 3, 4, 5]) # nlev (discrete)
    ]

num_samples = 100 # number of samples
lhs_samples = lhs_sampling(parameter_ranges, num_samples, discrete_params)

print(lhs_samples)
print(len(lhs_samples))

[[5.29835394e+08 2.56000000e+02 3.00000000e+00 6.21226656e+00
  8.21097377e-07 3.16008068e-02 6.81484815e+05 3.64438317e-07]
 [7.82976536e+08 6.40000000e+01 4.00000000e+00 7.79126645e+00
  9.76961510e-07 7.89911484e-02 3.05982663e+05 4.82387946e-07]
 [7.66202125e+08 2.56000000e+02 2.00000000e+00 5.41524634e+00
  4.95675339e-07 7.58777993e-02 7.09439336e+05 1.85054603e-07]
 [7.76152448e+08 8.00000000e+00 5.00000000e+00 6.67071643e+00
  1.36511648e-09 9.92958020e-02 5.69695994e+05 5.33337339e-08]
 [8.69886497e+08 1.28000000e+02 2.00000000e+00 3.09985258e+00
  7.85527914e-07 7.62934501e-02 6.78121875e+05 6.40738716e-07]
 [3.18307881e+07 1.60000000e+01 4.00000000e+00 9.53499878e+00
  7.18845870e-07 9.34507123e-02 9.08948590e+05 9.00778403e-07]
 [1.64825300e+07 2.56000000e+02 4.00000000e+00 3.25497524e+00
  3.87331854e-07 7.09259926e-02 2.97542212e+05 5.40918609e-07]
 [8.29352274e+08 3.20000000e+01 4.00000000e+00 6.98944041e+00
  4.28806814e-07 6.87531481e-03 4.27565393e+05 9.69460629e-07]


In [ ]:
def simulate_circuit(form, order, implementation, params):
    """
    Simulate the circuit and return the SNR value based on the selected
    form, order and implementation
    
    Args:
    - form (str): The (continuous) form of the filter ('FB', 'FF')
    - order (int): The order of the filter (2, 3, 4)
    - implementation (str): The implementation type ('Active-RC', 'Gm-C')
    - params (dict): A dictionary containing the parameters for the simulation
    
    Returns:
    - max_snr (float): The maximum SNR value obtained from the simulation
    """
    
    # define parameters:
    Bw = params.get('Bw')
    osr = params.get('osr')
    nlev = params.get('nlev')
    Hinf = params.get('Hinf')
    f0 = 0.
    tdac = [0, 1]
    
    # compute the sampling frequency:
    fs = Bw * osr * 2
    
    # synthesize the optimal NTF (Noise Transfer Function)
    ntf = cb.delsig.synthesizeNTF(order, osr, 2, Hinf, f0)
    
    # create analog frontend:
    dt = 1.0 / fs
    ABCDc, tdac2 = cb.delsig.realizeNTF_ct(ntf, form, tdac)
    analog_frontend = cb.AnalogFrontend.ctsdm(ABCDc, tdac2, dt, nlev)
    analog_frontend.dt = 1.0 / fs
    
    # run the SNR simulation (depending on the implementation):
    if implementation == 'Active-RC':
        max_snr = simulate_snr_active_rc(params, order, osr, analog_frontend)
    elif implementation == 'Gm-C':
        max_snr = simulate_snr_gm_c(params, order, osr, analog_frontend)    
    else:
        raise ValueError(f"Unknown implementation type: {implementation}")
    
    return max_snr



def simulate_snr_active_rc(params, order, osr, analog_frontend):
    """
    Simulate the SNR for Active-RC implementation
    
    Args:
    - params (dict): The parameter set
    - order (int): The order of the filter (2, 3, 4)
    - osr (int): The oversampling ratio
    
    Returns:
    - snr (float): The simulated SNR value.
    """
    
    # define parameters:
    Cint = np.ones(order) * params.get('Cint')
    gm = np.ones(order) * params.get('gm')
    Ro = np.ones(order) * params.get('Ro')
    Co = np.ones(order) * params.get('Co')
    
    # simulate SNR:
    try:
        ActiveRC_analog_frontend = cb.ActiveRC(
            analog_frontend, Cint, gm, Ro, Co
        )
                
        snr_ActiveRC, amp_ActiveRC, _ = ActiveRC_analog_frontend.simulateSNR(osr)
        max_snr_ActiveRC = max(snr_ActiveRC)
        
        if not np.isnan(max_snr_ActiveRC):
            print(f"Achieved SNR: {max_snr_ActiveRC} db")
            return max_snr_ActiveRC
        else:
            return None
        
    except (LinAlgError, ValueError) as e:
        print(f"Error with Active-RC parameters {params}: {e}")
        return None



# Function to simulate SNR for Gm-C
def simulate_snr_gm_c(params, order, osr, analog_frontend):
    """
    Simulate the SNR for Gm-C implementation
    
    Args:
    - params (dict): The parameter set
    - order (int): The order of the filter (2, 3, 4)
    - osr (int): The oversampling ratio
    
    Returns:
    - snr (float): The simulated SNR value
    """ 
    
    # define parameters:
    Cint = np.ones(order) * params.get('Cint')
    Ro = np.ones(order) * params.get('Ro')
    Cp = np.ones(order) * params.get('Cp')
    v_n = np.ones(order) * params.get('v_n')
    slew_rate = np.ones(order) * params.get('slew_rate')
    v_out_max = np.ones(order) * params.get('output_swing')
    v_out_min = -v_out_max
    
    # simulate SNR:
    try:
        GmC_analog_frontend = cb.GmC(
            analog_frontend, Cint, Ro, Cp, v_n, v_out_min, v_out_max, slew_rate
        )
    
        snr_GmC, amp_GmC, _ = GmC_analog_frontend.simulateSNR(osr)
        max_snr_GmC = max(snr_GmC)

        if not np.isnan(max_snr_GmC):
            print(f"Achieved SNR: {max_snr_GmC} db")
            return max_snr_GmC
        else:
            return None
    
    except (LinAlgError, ValueError) as e:
        print(f"Error with Gm-C parameters {params}: {e}")
        return None

In [6]:
# simulation test:

params = {'form': 'FB', 'order': 2, 'Bw': 20e3, 'osr': 64, 'nlev': 2.0, 'Hinf': 1.5, 'implementation': 'Active-RC', 'Cint': 1e-12, 'gm': 0.01, 'Ro': 1e4, 'Co': 1e-12}
snr = simulate_circuit('FB', 2, 'Active-RC', params)

/home/fgomez/.local/lib/python3.10/site-packages/scipy/signal/_ltisys.py:599: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  self.num, self.den = normalize(*system)
/home/fgomez/.local/lib/python3.10/site-packages/scipy/signal/_filter_design.py:1125: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  b, a = normalize(b, a)
INFO:root:Simulating continuous-time analog frontend for sinusoidal input


Achieved SNR: 73.90913681655965 db


In [7]:
def filter_parameter_relationships(params, implementation):
    """
    Apply relationships-based filters to ensure that the parameters 
    comply with physical and stability constraints

    Args:
    - params (dict): The parameters set
    - implementation (str): The implementation type ('Active-RC', 'Gm-C')

    Returns:
    - cond (bool): True if the parameters are valid, False otherwise
    """
    
    # add relationships between parameters as needed:
    
    """
    Example:
    
    if implementation == 'Active-RC':
        gm = params.get('gm')
        Ro = params.get('Ro')

        if gm * Ro > 1e3:
            return False # invalid relationship

    elif implementation == 'Gm-C':
        gm = params.get('gm')
        Ro = params.get('Ro')

        if gm * Ro < 1e-2:
            return False # invalid relationship
    """
    
    return True # if no issues, return True to indicate the parameters are valid

In [ ]:
# dataset generator test:

form = 'FB'
order = 2
implementation = 'Active-RC'
t_min = 600
num_samples = 50
min_snr = 50

# define parameter ranges (set as function args better?):
if implementation == 'Active-RC':
    parameter_ranges = [
        (10e3, 10e9), # Bw (Bandwidth)
        [8, 16, 32, 64, 128, 256], # osr (Oversampling Ratio) - Discrete
        [2, 3, 4, 5], # nlev (Number of Levels) - Discrete
        (1.0, 2.0), # Hinf (Infinity Gain)
        (100e-15, 5e-12), # Cint (Integration Capacitance)
        (1e-6, 1e-3), # gm (Transconductance)
        (100e3, 10e6), # Ro (Output Resistance)
        (50e-15, 1e-12), # Co (Output Capacitance)
    ]
elif implementation == 'Gm-C':
    parameter_ranges = [
        (10e3, 10e9), # Bw (Bandwidth)
        [8, 16, 32, 64, 128, 256], # osr (Oversampling Ratio) - Discrete
        [2, 3, 4, 5], # nlev (Number of Levels) - Discrete
        (1.0, 2.0), # Hinf (Infinity Gain)
        (100e-15, 2e-12), # Cint (Integration Capacitance)
        (10e3, 1e6), # Ro (Output Resistance)
        (1e-15, 100e-15), # Cp (Capacitor)
        (10e-9, 1e-6), # v_n (Noise Voltage)
        (1e-6, 100e-6), # slew_rate (Slew Rate)
        (0.5, 3.0), # output_swing (Output Swing)
    ]

# define discrete parameters:
discrete_params = [
    (1, [8, 16, 32, 64, 128, 256]), # osr (discrete)
    (2, [2, 3, 4, 5]) # nlev (discrete)
]

dataset = generate_dataset(form, order, implementation, t_min, num_samples, min_snr, parameter_ranges, discrete_params)
print(dataset.head())

/home/fgomez/.local/lib/python3.10/site-packages/scipy/signal/_ltisys.py:599: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  self.num, self.den = normalize(*system)
/home/fgomez/.local/lib/python3.10/site-packages/scipy/signal/_filter_design.py:1125: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  b, a = normalize(b, a)
INFO:root:Simulating continuous-time analog frontend for sinusoidal input
INFO:root:Simulating continuous-time analog frontend for sinusoidal input
/home/fgomez/.local/lib/python3.10/site-packages/scipy/signal/_ltisys.py:3137: ComplexWarning: Casting complex values to real discards the imaginary part
  yout[i, :] = (np.dot(system.C, xout[i, :]) +
/home/fgomez/.local/lib/python3.10/site-packages/scipy/signal/_ltisys.py:3141: ComplexWarning: Casting complex values to real discards the imaginary part
  yout[out_samples-1, :] = (np.dot(system.C, xout[out_samples-1, :])

Achieved SNR: 85.65987246814026 db
Error with Active-RC parameters {'Bw': np.float64(8891404496.865005), 'osr': 64, 'nlev': 2, 'Hinf': np.float64(1.0265453274097323), 'Cint': np.float64(4.131383834505862e-12), 'gm': np.float64(0.0008995552530749954), 'Ro': np.float64(4711341.325225656), 'Co': np.float64(1.6615539129108803e-13)}: The associated Hamiltonian pencil has eigenvalues too close to the imaginary axis


INFO:root:Simulating continuous-time analog frontend for sinusoidal input
INFO:root:Simulating continuous-time analog frontend for sinusoidal input


Error with Active-RC parameters {'Bw': np.float64(2625164570.8099756), 'osr': 16, 'nlev': 2, 'Hinf': np.float64(1.6922397729088927), 'Cint': np.float64(2.7685178563577413e-12), 'gm': np.float64(7.970307958471338e-05), 'Ro': np.float64(2815599.067788683), 'Co': np.float64(7.302186834445915e-13)}: Failed to find a finite solution.
Error with Active-RC parameters {'Bw': np.float64(2523902264.674548), 'osr': 16, 'nlev': 4, 'Hinf': np.float64(1.156314850251236), 'Cint': np.float64(8.042179535827527e-13), 'gm': np.float64(0.00016595914253259497), 'Ro': np.float64(2422038.894212398), 'Co': np.float64(1.6265847606058546e-13)}: The associated Hamiltonian pencil has eigenvalues too close to the imaginary axis


INFO:root:Simulating continuous-time analog frontend for sinusoidal input
INFO:root:Simulating continuous-time analog frontend for sinusoidal input


...

Error with Active-RC parameters {'Bw': np.float64(8927208388.337831), 'osr': 256, 'nlev': 4, 'Hinf': np.float64(1.688595595641322), 'Cint': np.float64(2.611749287884296e-12), 'gm': np.float64(0.0007704824864073333), 'Ro': np.float64(7038228.988680684), 'Co': np.float64(3.3567235893953275e-13)}: The associated Hamiltonian pencil has eigenvalues too close to the imaginary axis
Error with Active-RC parameters {'Bw': np.float64(4298321424.070264), 'osr': 32, 'nlev': 5, 'Hinf': np.float64(1.1162312318008407), 'Cint': np.float64(2.518032257802777e-12), 'gm': np.float64(0.00019394357147711278), 'Ro': np.float64(2077055.550872563), 'Co': np.float64(7.009630429906874e-13)}: The associated Hamiltonian pencil has eigenvalues too close to the imaginary axis


INFO:root:Simulating continuous-time analog frontend for sinusoidal input


Error with Active-RC parameters {'Bw': np.float64(1875069352.7801454), 'osr': 128, 'nlev': 2, 'Hinf': np.float64(1.2525250515245128), 'Cint': np.float64(2.1289415490172792e-12), 'gm': np.float64(0.0006642264616719581), 'Ro': np.float64(4027165.7526506926), 'Co': np.float64(1.0153431202905455e-13)}: Reordering of (A, B) failed because the transformed matrix pair (A, B) would be too far from generalized Schur form; the problem is very ill-conditioned. (A, B) may have been partially reordered.


INFO:root:Simulating continuous-time analog frontend for sinusoidal input
INFO:root:Simulating continuous-time analog frontend for sinusoidal input


Achieved SNR: 37.474013192899555 db
Error with Active-RC parameters {'Bw': np.float64(9719767406.230864), 'osr': 256, 'nlev': 3, 'Hinf': np.float64(1.0004588011869102), 'Cint': np.float64(3.693413328132927e-12), 'gm': np.float64(0.0009241774696876748), 'Ro': np.float64(9861990.632733008), 'Co': np.float64(8.023415263048468e-14)}: Failed to find a finite solution.
Error with Active-RC parameters {'Bw': np.float64(6521507417.245771), 'osr': 64, 'nlev': 5, 'Hinf': np.float64(1.2782439132252854), 'Cint': np.float64(1.1123468765853014e-12), 'gm': np.float64(0.000805114757594047), 'Ro': np.float64(2577196.2127847834), 'Co': np.float64(7.801874622897871e-13)}: The associated Hamiltonian pencil has eigenvalues too close to the imaginary axis

	Dataset saved as datasets/dataset_FB_2_Active-RC.csv

             Bw  osr  nlev      Hinf          Cint        gm            Ro  \
0  7.993443e+08  256     3  1.508113  1.192028e-12  0.000419  4.348632e+06   
1  2.326058e+08   32     4  1.565251  2.2648